# Experiment #2: Open vSwitch 4 Networks 2 Bridge Nodes 1 Shared Link

In this experiment we try to connect 4 networks with different hosts together with a bridge in the middle. We will be using Open vSwitch for the bridge, alternate solutions are Linux Bridge and Behavioral Model (BMv2).

## References
- [Open vSwitch Example](https://github.com/fabric-testbed/jupyter-examples/blob/main/fabric_examples/complex_recipes/openvswitch/openvswitch.ipynb)
- [BMv4](https://github.com/p4lang/behavioral-model)

## TODO's

- Add Open vSwitch flow configuratation on ports and so on.
- Add tools on how to track packets in the bridge.
- measure of congestion.


## Topology


<img src="./fig/Testbed-openvswitch-2.png" width=90%>

## Code

In [ ]:
from ipaddress import ip_address, IPv4Address, IPv4Network
import ipaddress
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager


fablib = fablib_manager()
fablib.show_config();

In [ ]:
slice_name= "Topology"

site1 = 'FIU'
print(f"Sites: {site1}")

site_node_count = 3
bridge1_name = 'bridge1'
bridge2_name = 'bridge2'
bridge3_name = 'bridge3'

In [9]:
slice = fablib.new_slice(name=slice_name)
default_image= 'default_ubuntu_22'

In [ ]:
bridge1 = slice.add_node(name=bridge1_name, site=site1, cores=2, ram=10, disk=50, image=default_image, host=f"{site1.lower()}-w1.fabric-testbed.net")
bridge1_nic1 = bridge1.add_component(model='NIC_Basic', name='nic_local_1')
bridge1_nic2 = bridge1.add_component(model='NIC_Basic', name='nic_local_2')
bridge1_nic3 = bridge1.add_component(model='NIC_Basic', name='nic_local_3')
bridge1_nic4 = bridge1.add_component(model='NIC_Basic', name='nic_local_4')


bridge2 = slice.add_node(name=bridge2_name, site=site1, cores=2, ram=10, disk=50, image=default_image, host=f"{site1.lower()}-w1.fabric-testbed.net")
bridge2_nic1 = bridge2.add_component(model='NIC_Basic', name='nic_local_1')
bridge2_nic2 = bridge2.add_component(model='NIC_Basic', name='nic_local_2')
bridge2_nic3 = bridge2.add_component(model='NIC_Basic', name='nic_local_3')
bridge2_nic4 = bridge2.add_component(model='NIC_Basic', name='nic_local_4')


bridge3 = slice.add_node(name=bridge3_name, site=site1, cores=2, ram=10, disk=50, image=default_image, host=f"{site1.lower()}-w1.fabric-testbed.net")
bridge3_nic1 = bridge3.add_component(model='NIC_Basic', name='nic_local_1')
bridge3_nic2 = bridge3.add_component(model='NIC_Basic', name='nic_local_2')

### Connecting bridges nodes 

In [ ]:
net = slice.add_l2network(name=f'net-br1-br2')
net.add_interface(bridge1.get_interfaces()[3])
net.add_interface(bridge2.get_interfaces()[3])

### Adding Nodes to bridge 1

In [ ]:
node_count = 3
for br_num in range(2):
    print(f"Adding nodes to {site1}-bridge{br_num+1}")
    disk_size = 500 if br_num == 0 else 50
    for node_num in range(node_count):
        cores = 2 if node_num > 0 else 10
        node_name = f"{site1.lower()}-bridge{br_num}-{node_num+1}"
        node = slice.add_node(name=node_name, site=site1, cores=cores, ram=32, disk=disk_size, image=default_image, host=f"{site1.lower()}-w2.fabric-testbed.net")
        iface = node.add_component(model='NIC_Basic', name='nic_local').get_interfaces()[0]    
        net = slice.add_l2network(name=f"net-br{br_num}-{node_num+1}")
        net.add_interface(iface)

        if br_num%2==0:
            net.add_interface(bridge1.get_interfaces()[node_num])
        else:
            net.add_interface(bridge2.get_interfaces()[node_num])
            
        if node_num==0:
            iface = node.add_component(model='NIC_Basic', name='nic_local2').get_interfaces()[0]
            net2 = slice.add_l2network(name=f"net-{br_num}-br3-{node_num+1}")
            net2.add_interface(iface)
            net2.add_interface(bridge3.get_interfaces()[br_num])

In [ ]:
print(slice)
slice.submit();

# OVS 

In [ ]:
try:
    for node in slice.get_nodes():
        if node.get_name().startswith("bridge"):
            stdout, stderr = node.execute('yes | sudo apt-get -y update && sudo apt-get upgrade', quiet=True) 
            stdout, stderr = node.execute('yes | sudo apt-get -y install openvswitch-switch openvswitch-common', quiet=True)
            stdout, stderr = node.execute('sudo apt-get -y install net-tools', quiet=True)
            print(f"done bridge: {node.get_name()}")
    print("Done")
except Exception as e:
    print(f"Exception: {e}")

In [ ]:
bridge1 = slice.get_node(name=bridge1_name)
stdout, stderr = bridge1.execute('sudo ovs-vsctl add-br br0')
for interface in bridge1.get_interfaces():
    stdout, stderr = bridge1.execute(f'sudo ovs-vsctl add-port br0 {interface.get_physical_os_interface_name()}')
    #Remove IP addresses for all interfaces
    stdout, stderr = bridge1.execute(f'sudo ifconfig {interface.get_physical_os_interface_name()} 0')
    
#bring the bridge up
stdout, stderr = bridge1.execute('sudo ifconfig br0 up')

print("Done")
stdout, stderr = bridge1.execute('sudo ovs-vsctl set bridge br0 stp_enable=true')
stdout, stderr = bridge1.execute('sudo ovs-appctl stp/show')


In [ ]:
bridge2 = slice.get_node(name=bridge2_name)
stdout, stderr = bridge2.execute('sudo ovs-vsctl add-br br0')
for interface in bridge2.get_interfaces():
    stdout, stderr = bridge2.execute(f'sudo ovs-vsctl add-port br0 {interface.get_physical_os_interface_name()}')
    #Remove IP addresses for all interfaces
    stdout, stderr = bridge2.execute(f'sudo ifconfig {interface.get_physical_os_interface_name()} 0')
    
#bring the bridge up
stdout, stderr = bridge2.execute('sudo ifconfig br0 up')

print("Done")
stdout, stderr = bridge2.execute('sudo ovs-vsctl set bridge br0 stp_enable=true')
stdout, stderr = bridge2.execute('sudo ovs-appctl stp/show')


In [ ]:
bridge3 = slice.get_node(name=bridge3_name)
stdout, stderr = bridge3.execute('sudo ovs-vsctl add-br br0')
for interface in bridge3.get_interfaces():
    stdout, stderr = bridge3.execute(f'sudo ovs-vsctl add-port br0 {interface.get_physical_os_interface_name()}')
    #Remove IP addresses for all interfaces
    stdout, stderr = bridge3.execute(f'sudo ifconfig {interface.get_physical_os_interface_name()} 0')
    
#bring the bridge up
stdout, stderr = bridge3.execute('sudo ifconfig br0 up')

print("Done")
stdout, stderr = bridge3.execute('sudo ovs-vsctl set bridge br0 stp_enable=true')
stdout, stderr = bridge3.execute('sudo ovs-appctl stp/show')

# Host Setup 

In [ ]:
shared = [
    "10.10.10.1/24",
    "10.10.10.2/24",
    "10.10.10.3/24",
    "10.10.10.4/24",
    "10.10.10.5/24",
    "10.10.10.6/24"
]

dedicated = [
    "10.10.10.7/24",
    "10.10.10.8/24",
]

for br_num in range(2):
    for i in range(3):
        host = slice.get_node(name=f'{site1.lower()}-bridge{br_num}-{i+1}')
        stdout, stderr = host.execute('sudo apt-get -y install net-tools', quiet=True)
        stdout, stderr = host.execute(f'sudo ip link set dev {host.get_interfaces()[0].get_physical_os_interface_name()} up', quiet=True)
        stdout, stderr = host.execute(f'sudo ip addr add {shared[i+3*br_num]} dev {host.get_interfaces()[0].get_physical_os_interface_name()}', quiet=True)
        if i == 0:
                stdout, stderr = host.execute(f'sudo ip link set dev {host.get_interfaces()[1].get_physical_os_interface_name()} up', quiet=True)
                stdout, stderr = host.execute(f'sudo ip addr add {dedicated[br_num]} dev {host.get_interfaces()[1].get_physical_os_interface_name()}', quiet=True)
                stdout, stderr = host.execute(f'sudo ip route add {dedicated[(br_num+1)%2]} dev {host.get_interfaces()[1].get_physical_os_interface_name()}', quiet=True)
                stdout, stderr = host.execute(f'sudo ip route add {shared[br_num]} dev {host.get_interfaces()[0].get_physical_os_interface_name()}', quiet=True)

In [ ]:
host1 = slice.get_node(name=f'{site1.lower()}-bridge0-1')
stdout, stderr = host1.execute('ping 10.10.10.2 -c 5')

In [ ]:
host6 = slice.get_node(name=f'{site1.lower()}-bridge0-1')
stdout, stderr = host1.execute('ping 10.10.10.5 -c 5')

In [ ]:
stdout, stderr = host1.execute('ping 10.10.10.3 -c 5')

In [18]:
slice.delete()